In [1]:
import sys
import json
import pandas as pd
import re
import csv
from elasticsearch import Elasticsearch, helpers

In [2]:
# Get started with elasticsearch
HOST = 'http://localhost:9200/'
es = Elasticsearch(hosts=[HOST])
# Check if elasticsearch is running
#!curl "http://localhost:9200/?pretty"

In [3]:
# Preprocessing the questions
qdf=pd.read_csv('questions.csv',error_bad_lines=False, warn_bad_lines=False , header=None, 
                index_col=0, parse_dates=[1],
                    quotechar='"', quoting=2)
# Clean up
# Drop lines in which the question field is empty
qdf=qdf.dropna(subset=[4])  
# Remove any bad indexes (containing letters)
good_indexes = [key for key in qdf.index if re.search('[a-zA-Z]', key) == None]
qdf = qdf.loc[good_indexes]
# Change indexes to integers
qdf.index=qdf.index.astype(int)
# Change column names
qdf.columns = ['date', 'userId', 'categoryId', 'question', 'description']
# Add the questionId column
qdf['questionId'] = qdf.index
# Combine question and description fields
qdf['question'] = qdf['question'].astype(str) + ' - ' + qdf['description'].astype(str)
qdf = qdf.drop(['description'], axis=1)
qdf['date'] = qdf['date'].apply(lambda x: x[:10])
qdf.head()

,date,userId,categoryId,question,questionId
0,,,,,
1,2009-01-28,1,32,Kunnen we volgende week weer schaatsen op natu...,1
2,2009-01-29,1,69,Wat is het lekkerste recept voor boerenkool? -...,2
3,2009-01-29,2,18,Hoeveel kleuren heeft de regenboog? - nan,3
4,2009-01-29,1,60,Wat is de symbolische betekenis van de kip die...,4
5,2009-01-29,3,66,waar kan ik in amsterdam het best een dwerg ha...,5


In [4]:
# Adding the correct category names
cdf=pd.read_csv('categories.csv',error_bad_lines=False, warn_bad_lines=False , header=None, 
                index_col=0, quotechar='"', quoting=2)
cdf.index=cdf.index.astype(int)
cdf[1] = cdf[1].astype(int)
cdf.columns = ['parent', 'name'] 

cdf['parent'] = cdf.apply(lambda x: cdf.iloc[int(x['parent'])-1]['name'], axis=1)
cdf['parent'][1] = ' '  # Apply special case to root category

qdf['category'] = qdf.apply(lambda x: str(cdf.loc[int(x['categoryId'])]['parent']) + ' - ' + str(cdf.loc[int(x['categoryId'])]['name']), axis=1)
qdf = qdf.drop(['categoryId'], axis=1)
qdf.head(10)

,date,userId,question,questionId,category
0,,,,,
1,2009-01-28,1,Kunnen we volgende week weer schaatsen op natu...,1,"Sport, Spel & Recreatie - Sporten"
2,2009-01-29,1,Wat is het lekkerste recept voor boerenkool? -...,2,Eten & Drinken - Koken & Recepten
3,2009-01-29,2,Hoeveel kleuren heeft de regenboog? - nan,3,Wetenschap - Natuur- en scheikunde
4,2009-01-29,1,Wat is de symbolische betekenis van de kip die...,4,Kunst & Cultuur - Beeldende kunst
5,2009-01-29,3,waar kan ik in amsterdam het best een dwerg ha...,5,Huis & Tuin - Huisdieren
6,2009-01-29,6,Waarom zie je nooit babyduifjes? - nan,6,Huis & Tuin - Overig
7,2009-01-29,7,Hoe krijg je een weggelopen konijn (ontsnapt) ...,7,Alle categorieën - Overig
8,2009-01-29,4,Wat is het synoniem voor synoniem ? - nan,8,Alle categorieën - Overig
9,2009-01-29,5,wat s de reden dat vogels niet vastvriezen aan...,9,Wetenschap - Biologie


In [5]:
# Indexing questions in elasticsearch
question_bulk = []
for index, row in qdf.iterrows():
    print(len(qdf) - index, end='\r') # Countdown timer
    question_bulk.append({'_type':'doc', 
                     '_index':'goeievragen',
                     'date':row['date'],
                     'userId':row['userId'],
                     'category':row['category'],
                     'question':row['question'],
                     'questionId':row['questionId']})


In [21]:
helpers.bulk(es,tuple(question_bulk))

(396045, [])

In [15]:
# Preprocessing answers
chunksize = 10 ** 3 # process chunks of 1000 documents
first_item = 1
colnames = range(8)
count = 0
for chunk in pd.read_csv('answers.csv',error_bad_lines=False, warn_bad_lines=False, header=None, 
                index_col=0, parse_dates=[1], quotechar='"', quoting=2, #low_memory=False,
                         chunksize=chunksize, names=colnames):
    # Perform the same cleaning process as with questions
    chunk = chunk.loc[chunk.index.dropna()] # drop empty indexes
    chunk = chunk.dropna(subset=[4]) # drop items with no answer text
    # Remove non-numeric indexes
    good_indexes = [key for key in chunk.index if re.search('[A-Za-z\WÀ-ÿ]', str(key)) == None]
    chunk = chunk.loc[good_indexes]
    # Remove non-numeric question indexes
    good_qids = [idx for idx, row in chunk.iterrows() if re.search('[A-Za-z\WÀ-ÿ]', str(row[3])) == None]
    chunk = chunk.loc[good_qids]
    # Remove non-numeric date indexes
    good_dates = [idx for idx, row in chunk.iterrows() if re.search('[A-Za-z\WÀ-ÿ]', str(row[1])) == None]
    chunk = chunk.loc[good_qids]
    # Change index type to integer
    chunk.index=chunk.index.astype(int)
    # Rename columns
    chunk.columns = ['date', 'userId', 'questionId', 'answer', 'thumbsDown', 'thumbsUp', 'bestAnswer']
    chunk['answerId'] = chunk.index # Add answerId column
    chunk['date'] = chunk['date'].apply(lambda x: x[:10]) # Truncate date to remove timestamp
    chunk = chunk.fillna(0) # Fill any empty fields with 0
    if first_item == 1: # add first chunk to result
        adf = chunk 
        first_item = 0
    else: # concatenate subsequent chunks
        adf = pd.concat([adf, chunk])
    count+=1
    print(count, end='\r') # Countdown timer

adf.head()

,date,userId,questionId,answer,thumbsDown,thumbsUp,bestAnswer,answerId
1,2009-01-29,3,2,met gepureerde aardappels natuurlijk!!! Duhhh ...,57,3,0,1
2,2009-01-29,1,8,"betekenisverwant, gelijkluidend",0,1,0,2
3,2009-01-29,8,3,"Kleuren van de regenboog : ROGGBIV (rood, oran...",0,5,1,3
4,2009-01-29,4,4,De schutters op het schilderij heetten kloveni...,0,6,1,4
5,2009-01-29,6,11,Van de eerste genoemde dieren is er slachtafva...,0,3,0,5


In [18]:
# Indexing answers in elasticsearch
answer_bulk = []
size = len(adf)
for index, row in adf.iterrows():
    answer_bulk.append({'_type':'doc', 
                     '_index':'goeievragen',
                     #'date':row['date'], # Causes parsing errors
                     'userId':row['userId'],
                     'questionId':row['questionId'],
                     'answer':row['answer'],
                     'thumbsDown':row['thumbsDown'],
                     'thumbsUp':row['thumbsUp'],
                     'bestAnswer':row['bestAnswer'],
                     'answerId':row['answerId']})
    size-=1
    print(size, end='\r') # Countdown timer

In [20]:
# Add the answer_bulk set to the index
helpers.bulk(es,tuple(answer_bulk))

(1401726, [])

In [33]:
# Some search functions
def match(term, category=False, jaar=False):
    # Simple search: category and year not indicated
    query={   "query": 
            { "multi_match" : 
            { "query": term, 
             "fields": [ "question", "answer", "category" ] 
    }}}
    if category: # Category is given
        query={ "query": 
        {"bool": { 
            "should": [
                { "match": { "question":   term        }}, 
                { "match": { "answer": term }}  
      ],
            "filter": [ { "term":  { "category": category }}]   
    }}}
    if jaar: # Year is given
        query={ "query": 
        {"bool": { 
            "should": [
                { "match": { "question":   term        }}, 
                { "match": { "answer": term }}  
      ],
            "filter": [ { "range": { "date": { "gte": jaar + "-01-01" }}}]   
    }}}
    if category and jaar: # Both are given
        query={ "query": 
        {"bool": { 
            "should": [
                { "match": { "question":   term        }}, 
                { "match": { "answer": term }}  
      ],
            "filter": [ { "term":  { "category": category }},
                        { "range": { "date": { "gte": jaar + "-01-01" }}}]   
    }}}
    
    res = es.search(index="goeievragen", body=query)
    print('Top ten results: ')
    for hit in res['hits']['hits']:
        print('-'*10)
        if 'answer' in hit["_source"]:
            print("Antwoord: %(answer)s" % hit["_source"])
        if 'question' in hit["_source"]:
            print("Categorie: %(category)s" % hit["_source"])
            print("Vraag: %(question)s" % hit["_source"])
        print("Datum: %(date)s" % hit["_source"])
        print("Id: %(questionId)s" % hit["_source"])
        
def get_answers(questionId):
    query={"query": {"term" : { "questionId": questionId }}}
    res = es.search(index="goeievragen", body=query)
    for hit in res['hits']['hits']:
        if 'answer' in hit["_source"]:
            print("Antwoord: %(answer)s" % hit["_source"])
    

In [36]:
#match('tijd', 'wetenschap')
match('tijd', 'wetenschap', '2015')
#get_answers(7078)

Top ten results: 
----------
Categorie: Wetenschap - Overig
Vraag: Wat is tijd? - Wat is tijd volgens bergson
Datum: 2009-03-26
Id: 1882
----------
Categorie: Wetenschap - Overig
Vraag: wat is het tegenovergestelde van \TIJD\" ? of is tijd de enige constante en is tijd iets dat altijd verstrijkt.....?" - nan
Datum: 2011-09-26
Id: 244747
----------
Categorie: Wetenschap - Overig
Vraag: Hoe kan er tijd zijn, als die niet gemeten wordt? - Tijd is op zich toch een illusie? Wij hebben tijd, omdat wij hem meten. Als hij niet gemeten wordt, is er geen tijd en net daardoor alle tijd van de wereld. Toch?
Datum: 2011-06-14
Id: 198879
----------
Categorie: Wetenschap - Overig
Vraag: Het is niet de big bang dat tijd heeft gemaakt.Het is logisch dat er tijd is,omdat naarmate iets langer dan een moment bestaat er dan tijd verstrijkt - Door de big bang ontstond heelal en (tijd).\
Maar het is toch logisch dat (tijd) bestaat, omdat als iets langer dan een moment bestaat, het dan tijd in beslag neemt.
D